# Load Data

In [5]:
import os
import sys
import pandas as pd

folder_path = os.path.join(os.path.dirname(os.getcwd()), 'Data_Test_v1')
print(folder_path)
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv','Supplier_Data.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]
supplier_data = data_frames[4]

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\Data_Test_v1


In [7]:
import numpy as np

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# Parameters
price = 550
hold = salvage = 50

num_suppliers = len(supplier_data)
costs = supplier_data['price'].values
max_capacity = supplier_data['max'].values
min_capacity = supplier_data['min'].values
fixed_cost = supplier_data['fixed'].values



# Calculation of quantiles for the newsvendor problem
critical_ratio = []
for i in range(num_suppliers):
    cr = (price - costs[i]) / ((costs[i] - salvage) + (price - costs[i]))
    critical_ratio.append(round(cr,4))
print(critical_ratio)


# Number of simulations for estimating expected profit
num_simulations = 1000  # Number of simulations for estimating expected profit

[0.092, 0.196, 0.186, 0.202, 0.112, 0.194]


# ANN

In [68]:
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.stats import reciprocal

total_capacity = 260000
n_outputs = len(target_train.columns)

# Define a custom loss function for the newsvendor problem with quantile regression
def newsvendor_quantile_loss(q, min, max, fixed, y_true, y_pred):
    q = tf.constant(np.array(q), dtype=tf.float32)
    e = y_true - y_pred
    base_loss = tf.reduce_mean(tf.maximum(q*e, (q-1)*e), axis=-1) # Quantile loss of critical ratio

    


    # Calculate the total demand
    total_demand = tf.reduce_sum(y_pred, axis=-1)

    # Calculate the penalty
    penalty = tf.maximum(0.0, total_demand - total_capacity)
    
    # Add the penalty to the base loss
    loss = base_loss + penalty
    
    return loss

# Define a function to create a model
def create_model(n_hidden, n_neurons, learning_rate, activation, input_shape=[3]): #learning_rate_input=3e-3, 
    model = Sequential()
    model.add(Dense(n_neurons, activation=activation, input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation=activation))
    model.add(Dense(n_outputs))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss=lambda y,f: newsvendor_quantile_loss(critical_ratio,y,f), optimizer=optimizer)
    return model

# Define a function to build the model
def model_builder(n_hidden=1, n_neurons=30, learning_rate=3e-3 , activation = 'relu'): #
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons, learning_rate=learning_rate, activation=activation)

# Create a model
model_ANN = model_builder()

# Define the parameter grid
param_distribs = {
    "n_hidden": [1, 3, 5, 7, 9],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [8,16, 32, 64],
    "epochs": [10, 20, 30, 40, 50],
    "activation": ['relu']
}

# Optimize the model using RandomizedSearchCV
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distribs, n_iter=20, cv=5, scoring='neg_mean_squared_error')

# Fit the model
target_train = target_train.astype('float32')
rnd_search_cv_ANN.fit(data_train, target_train)
print(rnd_search_cv_ANN.best_params_)

# Make predictions
target_pred_ANN = rnd_search_cv_ANN.predict(data_test)
target_pred_ANN = pd.DataFrame(np.row_stack(target_pred_ANN))

# Calculate the MSE
mse = mean_squared_error(target_test, target_pred_ANN)
print("MSE:", mse)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 16, 'epochs': 40, 'learning_rate': 0.0002658207530307782, 'n_hidden': 9, 'n_neurons': 42}
MSE: 635089404.219889


# DT

In [69]:
from sklearn.ensemble import RandomForestRegressor

# Create a random forest regressor
forest = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model
forest.fit(data_train, target_train)

# Make predictions
target_pred_forest = forest.predict(data_test)

# Calculate the MSE
mse = mean_squared_error(target_test, target_pred_forest)
print("MSE:", mse)

print(target_pred_forest)

target_pred_forest = pd.DataFrame(np.row_stack(target_pred_forest))

MSE: 12490019.539466674
[[153127.77  67776.74  32669.28]
 [188211.19  68300.52  33204.41]
 [182258.83  68341.08  33216.87]
 [199972.17  68203.36  33329.18]
 [213056.91  68183.45  33155.59]
 [125226.08  68308.16  33187.65]
 [142511.53  68269.48  33072.42]
 [170220.48  68150.79  33177.51]
 [156661.1   68181.6   33184.26]
 [212622.72  68158.28  33252.52]
 [191576.95  68160.4   33219.24]
 [195603.2   68296.14  33336.13]
 [143181.03  63958.35  31170.34]
 [179665.11  68223.59  33268.21]
 [206768.73  68396.97  33251.77]
 [153312.8   68337.39  33467.76]
 [181095.83  68151.45  33215.09]
 [180811.02  68148.2   33339.77]
 [151903.63  68291.66  33353.43]
 [226785.81  68102.46  33198.9 ]
 [131918.77  68357.44  33061.63]
 [166095.92  68209.34  33196.01]
 [192804.21  68109.27  33194.25]
 [244346.59  68257.28  33252.76]
 [200272.79  68098.52  33237.41]
 [227661.34  68204.25  33271.25]
 [158117.62  68274.29  33337.07]
 [224779.64  68106.96  33359.73]
 [244894.97  68354.12  33151.15]
 [189039.21  68147.

In [70]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor

# Specify the parameter grid for RandomizedSearchCV
param_distribs = {
    'estimator__boosting_type': ['gbdt', 'dart', 'goss'],
    'estimator__num_leaves': [10, 20, 30, 40, 50],
    'estimator__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'estimator__feature_fraction': [0.8, 0.9, 1.0],
    'estimator__bagging_fraction': [0.8, 0.9, 1.0],
    'estimator__bagging_freq': [3, 4, 5, 6, 7],
}

cr = np.mean(critical_ratio)

# Create the LightGBM model
model = lgb.LGBMRegressor(objective='quantile', alpha=cr, metric='quantile', verbose=0)

# Wrap the model with MultiOutputRegressor
model = MultiOutputRegressor(model)

# Perform RandomizedSearchCV
rnd_search = RandomizedSearchCV(model, param_distributions=param_distribs, n_iter=10, cv=3)

# Fit the model
rnd_search.fit(data_train, target_train)
print(rnd_search.best_params_)

# Get the best model
best_model = rnd_search.best_estimator_

# Predict the demand using the best model
target_pred_DT = best_model.predict(data_test)
target_pred_DT = pd.DataFrame(np.row_stack(target_pred_DT))
print(target_pred_DT)

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
9 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\la

# Costs

In [71]:
# Loop over each week in target_test
overall_costs_ANN = 0
overall_costs_DT = 0

for i in range(len(target_test)):
    for j in range(len(target_test.columns)):
        # Calculate understock and overstock costs
        cost_ANN = 0
        cost_DT = 0

        if target_pred_ANN.iloc[i, j] < target_test.iloc[i, j]:
            cost_ANN = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(target_pred_ANN.iloc[i, j]))

        if target_pred_ANN.iloc[i, j] > target_test.iloc[i, j]:
            cost_ANN = (costs[j] - salvages[j]) * (np.round(target_pred_ANN.iloc[i, j]) - target_test.iloc[i, j])

        if target_pred_DT.iloc[i, j] < target_test.iloc[i, j]:
            cost_DT = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(target_pred_DT.iloc[i, j]))

        if target_pred_DT.iloc[i, j] > target_test.iloc[i, j]:
            cost_DT = (costs[j] - salvages[j]) * (np.round(target_pred_DT.iloc[i, j]) - target_test.iloc[i, j])

        # Calculate the total costs for the week
        overall_costs_ANN += cost_ANN
        overall_costs_DT += cost_DT

# Print the overall costs
print('Overall costs for ANN: ', int(overall_costs_ANN))
print('Overall costs for DT: ', int(overall_costs_DT))

Overall costs for ANN:  5391960
Overall costs for DT:  712698
